<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5c08925b72899125744f66501b883c7603f486a137b3bb28750777eb32c549a2
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-22 10:11:14
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 89.82 K (0.64%)
Current PnL: -25.90 L (-16.73%)
CY Booked + Current PnL: -11.61 L (-7.5%)
-------------------
Total profit:  1.50 L
Total loss:  -27.40 L
-------------------
Total Booked + Current PnL: 15.37 L (12.02%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.12%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.54 L (64.14%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.08,-19.57,24.44,0.09,19810.0,-19721.0,81056.0,88.05,32.0,M-SC,2.27,253.0,-1.00,0.58,6.49,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.57,8.22,10.60,19.69,21996.0,15768.0,207510.0,-2.32,54.0,X-MC,2.46,78.0,0.72,1.48,15.58,XY25,NTT,AC
50,MASFIN,398.61,-0.41,-3.66,26.68,22.05,25185.0,-3585.0,94395.0,-16.89,50.0,H-SC,6.47,164.0,-0.14,0.67,37.21,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.18,-15.41,18.24,0.02,26933.0,-26900.0,147659.0,125.45,44.0,M-SC,12.65,234.0,-1.00,1.05,28.44,OX40N,NTT,PAINTS
43,ITC,452.00,0.39,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.84,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,3.78,-18.44,84.88,50.79,97368.0,-25941.0,114712.0,4.75,72.0,H-SC,7.25,151.0,-0.27,0.82,33.56,XR,ATH,IT
7,ATULAUTO,844.00,3.57,-23.60,90.54,45.57,136356.0,-46529.0,150603.0,3251.35,47.0,M-SC,5.03,244.0,-0.34,1.07,6.77,XY24,NTT,AUTO
21,DEN,75.00,2.94,-39.06,133.14,42.07,68530.0,-32992.0,51472.0,83.94,55.0,M-SC,7.82,235.0,-0.48,0.37,9.31,AR,NTT,ENTERTAINMENT
29,HAPPSTMNDS,1480.71,2.44,-38.14,198.83,84.85,157632.0,-48885.0,79280.0,-26.54,50.0,H-SC,13.55,147.0,-0.31,0.56,4.12,AR,ATH,IT
41,INFY,1972.00,2.18,13.99,17.77,34.24,63376.0,43759.0,356647.0,-11.34,74.0,X-LC,5.30,2.0,0.69,2.54,23.43,X40,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.00,-0.89,-35.64,55.35,-0.01,45530.0,-45547.0,82258.0,16726.09,35.0,M-SC,10.42,233.0,-1.00,0.59,5.28,OX40N,NTT,AC
51,MEDANTA,1486.00,-0.65,-2.98,27.88,24.08,34340.0,-3778.0,123172.0,-10.78,41.0,X-SC,5.89,89.0,-0.11,0.88,16.06,XY24,NTT,HEALTHCARE
35,ICICIGI,2252.93,-0.60,2.90,15.97,19.34,32885.0,5807.0,205916.0,-18.85,42.0,X-MC,4.68,68.0,0.18,1.47,19.30,X40,ATH,INSURANCE
19,COLPAL,3726.84,-0.53,-20.27,77.48,41.51,162700.0,-53375.0,209990.0,-4.98,39.0,X-MC,7.69,61.0,-0.33,1.49,0.60,XY25,ATH,FMCG
68,STARHEALTH,761.00,-0.43,-10.34,64.24,47.26,161925.0,-29061.0,252062.0,22.40,40.0,H-SC,11.18,171.0,-0.18,1.79,35.05,XY24,NTT,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.7,0.41,107.07,107.91,158725.0,600.0,148244.0,-18.12,63.0,M-SC,12.0,216.0,0.0,1.06,10.53,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.70,0.41,107.07,107.91,158725.0,600.0,148244.0,-18.12,63.0,M-SC,12.00,216.0,0.00,1.06,10.53,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.18,-15.41,18.24,0.02,26933.0,-26900.0,147659.0,125.45,44.0,M-SC,12.65,234.0,-1.00,1.05,28.44,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.08,-19.57,24.44,0.09,19810.0,-19721.0,81056.0,88.05,32.0,M-SC,2.27,253.0,-1.00,0.58,6.49,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,0.95,-21.24,67.63,32.03,70224.0,-28000.0,103835.0,-14.69,64.0,H-SC,9.33,148.0,-0.40,0.74,16.30,OX40N,NTT,IT
66,SIS,528.00,-0.06,-23.59,58.70,21.26,49800.0,-26194.0,84838.0,2002.07,53.0,H-SC,4.72,166.0,-0.53,0.60,14.63,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.23,8.47,58.82,72.27,108107.0,14345.0,183793.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.31,15.16,XR,NTT,IT
25,FINCABLES,1641.55,0.70,0.41,107.07,107.91,158725.0,600.0,148244.0,-18.12,63.0,M-SC,12.00,216.0,0.00,1.06,10.53,OX40N,ATH,CABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.23,8.47,58.82,72.27,108107.0,14345.0,183793.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.31,15.16,XR,NTT,IT
25,FINCABLES,1641.55,0.70,0.41,107.07,107.91,158725.0,600.0,148244.0,-18.12,63.0,M-SC,12.00,216.0,0.00,1.06,10.53,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.12,-4.86,117.28,106.72,137619.0,-5994.0,117342.0,-52.92,36.0,H-SC,3.96,139.0,-0.04,0.84,17.38,AR,ATH,MISC
6,ASIANTILES,137.00,0.82,-3.67,88.60,81.67,80063.0,-3446.0,90364.0,8133.33,72.0,L-SC,11.56,271.0,-0.04,0.64,74.99,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,0.21,-2.83,39.57,35.63,80585.0,-5922.0,203652.0,-16.74,42.0,H-MC,2.51,119.0,-0.07,1.45,13.37,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.82,-17.69,88.89,55.47,178004.0,-43034.0,200252.0,-26.96,33.0,X-MC,14.79,64.0,-0.24,1.43,2.61,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.06,-38.71,120.61,35.20,95111.0,-49812.0,78858.0,3.39,35.0,X-SC,16.75,92.0,-0.52,0.56,1.15,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-0.27,-17.25,73.19,43.32,98645.0,-28092.0,134780.0,-28.92,35.0,X-SC,5.42,90.0,-0.28,0.96,3.49,X40N,ATH,ELECTRICAL
33,HINDUNILVR,2922.00,0.26,-11.07,27.82,13.67,63597.0,-28463.0,228600.0,-17.78,35.0,X-LC,2.71,14.0,-0.45,1.63,7.92,XY25,NTT,FMCG
3,ACC,3906.00,0.80,-25.80,121.24,64.15,214049.0,-61401.0,176550.0,-56.75,36.0,X-SC,3.62,82.0,-0.29,1.26,0.80,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.20,-0.47,25.07,24.48,62842.0,-1188.0,250668.0,-81.09,48.0,X-SC,0.95,86.0,-0.02,1.78,22.56,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.34,-3.83,25.96,21.14,56759.0,-8700.0,218639.0,-10.57,41.0,X-MC,1.13,74.0,-0.15,1.56,23.24,XY24,NTT,PAINTS
20,DABUR,735.0,0.04,-3.39,48.74,43.70,116812.0,-8405.0,239663.0,-11.44,36.0,X-MC,2.05,72.0,-0.07,1.71,11.15,XY24,BTT,FMCG
78,UNITDSPR,1644.0,0.31,3.66,16.50,20.77,40047.0,8565.0,242709.0,-5.34,46.0,X-MC,2.13,62.0,0.21,1.73,10.83,X40N,NTT,BREWERIES
43,ITC,452.0,0.39,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.84,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-0.53,-20.27,77.48,41.51,162700.0,-53375.0,209990.0,-4.98,39.0,X-MC,7.69,61.0,-0.33,1.49,0.60,XY25,ATH,FMCG
3,ACC,3906.00,0.80,-25.80,121.24,64.15,214049.0,-61401.0,176550.0,-56.75,36.0,X-SC,3.62,82.0,-0.29,1.26,0.80,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-0.06,-38.71,120.61,35.20,95111.0,-49812.0,78858.0,3.39,35.0,X-SC,16.75,92.0,-0.52,0.56,1.15,X40,NTT,FOOTWEAR
53,PAGEIND,51605.08,2.00,-9.49,41.54,28.11,75727.0,-19110.0,182300.0,-34.38,36.0,X-MC,14.03,55.0,-0.25,1.30,2.14,X40,ATH,APPARELS
54,PGHH,17907.65,0.70,-3.75,39.41,34.19,80995.0,-8000.0,205520.0,-32.38,48.0,X-MC,5.21,57.0,-0.10,1.46,2.25,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.19,-29.72,103.25,42.84,47171.0,-19320.0,45686.0,-53.72,44.0,X-SC,34.59,83.0,-0.41,0.33,3.36,XY24,NTT,MISC
59,RELAXO,1176.00,1.06,-47.37,189.37,52.31,144684.0,-68757.0,76403.0,-43.96,46.0,X-SC,7.25,91.0,-0.48,0.54,2.27,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.06,-38.71,120.61,35.20,95111.0,-49812.0,78858.0,3.39,35.0,X-SC,16.75,92.0,-0.52,0.56,1.15,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.65,-2.98,27.88,24.08,34340.0,-3778.0,123172.0,-10.78,41.0,X-SC,5.89,89.0,-0.11,0.88,16.06,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.27,-17.25,73.19,43.32,98645.0,-28092.0,134780.0,-28.92,35.0,X-SC,5.42,90.0,-0.28,0.96,3.49,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.59,-9.23,32.98,20.71,113229.0,-34912.0,343325.0,-21.68,71.0,X-LC,3.67,1.0,-0.31,2.44,14.72,X40,ATH,IT
41,INFY,1972.00,2.18,13.99,17.77,34.24,63376.0,43759.0,356647.0,-11.34,74.0,X-LC,5.30,2.0,0.69,2.54,23.43,X40,NTT,IT
75,TMPV,600.00,1.19,-26.86,68.14,22.98,112582.0,-60676.0,165222.0,-24.30,47.0,X-LC,6.58,3.0,-0.54,1.18,3.92,XY24,NTT,AUTO
81,VBL,671.64,1.43,-3.83,41.07,35.67,124751.0,-12090.0,303752.0,-15.30,54.0,X-LC,3.81,4.0,-0.10,2.16,9.50,X40N,ATH,FMCG
43,ITC,452.00,0.39,-1.33,12.27,10.78,29096.0,-3187.0,237131.0,-41.15,49.0,X-LC,2.16,5.0,-0.11,1.69,3.84,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.28,-37.46,112.50,32.89,54317.0,-28924.0,48282.0,-696.43,54.0,L-MC,6.55,259.0,-0.53,0.34,32.96,XR,NTT,BANKS
56,RAJESHEXPO,518.00,0.86,-60.18,151.31,0.07,82649.0,-82555.0,54622.0,1902.53,67.0,L-SC,2.74,268.0,-1.00,0.39,35.85,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,0.82,-3.67,88.60,81.67,80063.0,-3446.0,90364.0,8133.33,72.0,L-SC,11.56,271.0,-0.04,0.64,74.99,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.41,-3.66,26.68,22.05,25185.0,-3585.0,94395.0,-16.89,50.0,H-SC,6.47,164.0,-0.14,0.67,37.21,XR,ATH,FINANCE
13,BSOFT,831.70,3.78,-18.44,84.88,50.79,97368.0,-25941.0,114712.0,4.75,72.0,H-SC,7.25,151.0,-0.27,0.82,33.56,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,3.78,-18.44,84.88,50.79,97368.0,-25941.0,114712.0,4.75,72.0,H-SC,7.25,151.0,-0.27,0.82,33.56,XR,ATH,IT
61,ROUTE,2227.21,0.45,-45.11,207.99,69.05,151912.0,-60028.0,73038.0,-55.56,57.0,H-SC,21.31,140.0,-0.40,0.52,12.64,SR,ATH,IT
6,ASIANTILES,137.00,0.82,-3.67,88.60,81.67,80063.0,-3446.0,90364.0,8133.33,72.0,L-SC,11.56,271.0,-0.04,0.64,74.99,XR,NTT,CERAMICS
84,WIPRO,420.00,0.23,8.47,58.82,72.27,108107.0,14345.0,183793.0,-6.99,70.0,M-LC,3.06,99.0,0.13,1.31,15.16,XR,NTT,IT
41,INFY,1972.00,2.18,13.99,17.77,34.24,63376.0,43759.0,356647.0,-11.34,74.0,X-LC,5.30,2.0,0.69,2.54,23.43,X40,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.87
2,50,76.59


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.98
MC    30.81
LC    24.25
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.77
X40      23.09
X40N     12.99
XR        9.84
XY25      9.03
AR        8.95
OX40N     7.60
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.32
H-SC    23.74
X-LC    20.81
M-SC    11.69
X-SC     8.03
H-MC     4.81
L-SC     1.52
M-MC     1.34
M-LC     1.31
H-LC     1.11
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.24,-7.75,42.39
IT,13.21,-13.37,70.37
FINANCE,9.48,-19.67,71.71
MISC,7.26,-27.31,79.38
ELECTRICAL,6.00,-11.36,52.15
PAINTS,5.80,-13.32,30.15
INSURANCE,4.77,-1.29,35.95
PHARMA,4.08,-2.54,34.81
AUTO,2.87,-30.07,75.17


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3162953.0,21
AR,1315503.0,10
XR,1279809.0,13
X40,980464.0,14
X40N,860945.0,9
OX40N,725579.0,10
XY25,373255.0,6
SR,277921.0,2
MH,77150.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3643466.0,25
M-SC,1417845.0,15
X-MC,1412740.0,16
X-LC,803541.0,11
X-SC,791851.0,8
H-MC,402605.0,3
L-SC,248094.0,3
M-LC,108107.0,1
H-LC,77722.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1274656.0      6
           AR         903733.0      5
M-SC       XY24       810459.0      6
H-SC       XR         777556.0      7
X-MC       X40        486661.0      7
           XY24       403355.0      3
           X40N       338028.0      4
H-SC       OX40N      332450.0      4
M-SC       OX40N      310480.0      5
X-SC       X40N       306171.0      3
X-LC       X40        303683.0      5
X-SC       XY24       295560.0      3
H-SC       SR         277921.0      2
X-LC       X40N       216746.0      2
H-MC       AR         211526.0      2
           XY24       191079.0      1
X-SC       X40        190120.0      2
X-LC       XY24       187844.0      2
X-MC       XY25       184696.0      2
M-SC       XR         174384.0      2
L-SC       XR         165445.0      2
M-SC       AR         122522.0      2
M-LC       XR         108107.0      1
X-LC       XY25        95268.0      2
L-SC       OX40N       82649.0      1
H-LC       AR          77722.0      1
H-SC       MH          77150.0      1
M-MC       XY25        56534.0      1
L-MC       XR          54317.0      1
L-LC       XY25        36757.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
